## **Imports**

First, it's necessary to import the libraries that will be used in the script below and the dataset obtained from the [website](https://sistemas.anatel.gov.br/se/public/view/b/licenciamento.php) of the Brazilian National Telecommunications Agency.

### **Libraries**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
import math
import zipfile
import os

font_dir = '/usr/share/fonts/truetype/msttcorefonts/'
fm.fontManager.addfont(font_dir + 'Times_New_Roman.ttf')
plt.rcParams['font.family'] = 'Times New Roman'

### **Dataset**

The dataset columns used in this scripts are:
* **'NumEstacao':** unique number that characterizes a station;
* **'DataValidade':** expiration date of that radio frequency associated with the station;
* **'Azimute':** positioning in degrees in relation to the north of the antenna radiation main lobe;
* **'AnguloMeiaPotenciaAntena':** half power angle in degrees.

In [2]:
#The variable path must contain a string with the path to the dataset used.

#path = '/home/oai-ufrn/Repositories/nir-measurement-methodology/datasets/natal_04_11_2024.csv' #2024
#year = '2024'

path = '/home/oai-ufrn/Repositories/nir-measurement-methodology/datasets/natal_03_31_2025.csv' #2025
year = '2025'

results_path = '/home/oai-ufrn/Repositories/nir-measurement-methodology/results/' + str(year) +'/azimuth_analysis'
os.makedirs(results_path, exist_ok=True) 

df = pd.read_csv(path, encoding='unicode_escape')

## **Data Adjustment**

The cell below creates a dataframe with the stations selected in previous steps of the methodology, it must be changed with the station numbers of the scenario to be evaluated.

#2024
filter = {
    'NumStation': [
        922234, 922242, 5187354, 6556752, 431377928, 441068146, 665600062, 686142519,
        686143205, 686696786, 690004206, 690904738, 690905823, 690905866, 690905874,
        690905912, 690905939, 691182450, 691182701, 691182728, 691182760, 691182850,
        691182930, 691182957, 691183031, 691183082, 691223149, 691346259, 691347646,
        691348090, 691571643, 692295992, 692700170, 692817107, 692914501, 693190442,
        695207075, 695364405, 695561146, 695561219, 695706616, 698154673, 698178238,
        1001843867, 1002291736, 1003228795, 1003326657, 1004211527, 1005215631, 1005343001,
        1006733571, 1006918474, 1008016842, 1008016958, 1008246368, 1008246376, 1008690713,
        1008779110, 1009580156, 1010052729, 1015604703, 1015630488
]}
filter = pd.DataFrame(filter)

In [3]:
#2025
filter = {
    'NumStation': [
        907936, 922234, 922242, 949540, 4861469, 5187354, 6556752, 431377880, 431377898,
        431377910, 431377928, 441068146, 665596960, 665599552, 665643896, 665756810, 665756836,
        665756852, 665756860, 665756895, 683585746, 686142519, 686143205, 686696786, 690004206,
        690904738, 690905823, 690905866, 690905874, 690905912, 690905939, 691182450, 691182701,
        691182728, 691182760, 691182850, 691183031, 691183082, 691346259, 691347646, 691571643,
        691986827, 692295992, 692914501, 695207075, 695561146, 695561219, 695706616, 698178238,
        1001843867, 1003228558, 1003228795, 1003326657, 1004211527, 1005343001, 1006733571,
        1006918474, 1008016842, 1008016958, 1008246368, 1008246376, 1008779110, 1015098069,
        1015604703, 1015630488, 1015804915, 1016093370, 1016098542, 1016116109, 1016158405,
        1016159665, 1016224459, 1016257594, 1016273476, 1016305017, 1016305025, 1016343750,
        1016356339, 1016487565, 1016540237, 1016560181, 665600062
]}
filter = pd.DataFrame(filter)

Some columns are not in the a format usable for the script, so they have been fixed. Rows whose 'DataValidade' value is prior to the year the work was developed need to be removed.

In [4]:
#Conversion of the column 'DataValidade' to the datetime64[ns] type.
df['DataValidade'] = pd.to_datetime(df['DataValidade'])

#Exclusion of lines of 'DataValidade' prior to December 31, 2023.
df = df[df['DataValidade'] > pd.to_datetime(str(year)+'-01-01', format='%Y-%m-%d')]

#Filtering of the ANATEL's dataset based on the stations manually selected.
df_filtered = df.loc[df['NumEstacao'].isin(filter['NumStation'])].copy()

#Correction of the data type of the columns 'Azimute' and 'AnguloMeiaPotenciaAntena'.
df_filtered['Azimute'] = pd.to_numeric(df['Azimute'], errors='coerce')
df_filtered['Azimute'] = df_filtered['Azimute'].astype(int)
df_filtered['AnguloMeiaPotenciaAntena'] = pd.to_numeric(df['AnguloMeiaPotenciaAntena'], errors='coerce')

To visualize the worst coverage scenario, only the lowest value of the half-power angle must to be used in the plots. The script below arrange the data in a list of tuples, which contain: 
1) The station number;
2) The station azimuths;
3) The smallest half-power angles for each azimuth. 

In [5]:
#Groups the data by 'NumEstacao' and 'Azimute' and selects the lowest value of 'AnguloMeiaPotenciaAntena'.
min_values = df_filtered.groupby(['NumEstacao', 'Azimute'])['AnguloMeiaPotenciaAntena'].min().reset_index()

#Finds the minimum non-zero value in 'AnguloMeiaPotenciaAntena'.
for index, row in min_values.iterrows():
    if row['AnguloMeiaPotenciaAntena'] <= 0:
        next_min = df_filtered[(df_filtered['NumEstacao'] == row['NumEstacao']) & (df_filtered['Azimute'] == row['Azimute']) & (df_filtered['AnguloMeiaPotenciaAntena'] > 0)]['AnguloMeiaPotenciaAntena'].min()
        if not math.isnan(next_min):
            min_values.at[index, 'AnguloMeiaPotenciaAntena'] = next_min

#Group by 'NumEstacao' and build the list of azimuths and minimum half-power angle values, forming a list of tuples.
df_stations_info = []
for station in min_values['NumEstacao'].unique():
    temp = min_values[min_values['NumEstacao'] == station]
    azimuths = temp['Azimute'].tolist()
    half_power_angles = temp['AnguloMeiaPotenciaAntena'].tolist()
    df_stations_info.append((station, azimuths, half_power_angles))

In case of multi-tenant cell situations, the information of each station must to be grouped, so the function below was designed to do this grouping.

In [6]:
def group_stations(info, *stations):
    #Check if at least two stations are to be grouped.
    if len(stations) < 2:
        return "At least two stations are required for merging."

    #Find data for each station in the list.
    station_data = []
    for station in stations:
        station_info = None
        for data in info:
            if data[0] == station:
                station_info = data
                break
        if station_info is None:
            return f"Station {station} not found in the configuration list."
        station_data.append(station_info)

    #Combine station data.
    new_azimuths = [azimuth for data in station_data if data[1] is not None for azimuth in data[1]]
    new_half_power_angles = [angle for data in station_data if data[2] is not None for angle in data[2]]

    #Create the number of the new station.
    new_station_number = ' and '.join(map(str, stations))

    #Remove original stations from the list.
    for station in station_data:
        info.remove(station)

    #Add the new combined station to the list.
    info.append((new_station_number, new_azimuths, new_half_power_angles))

    #Return the updated list.
    return info

The cell below creates a dataframe with the combinations of ERBs that share the same infrastructure. Each column of 'combinations' is an association of different Station Numbers, it was also formulated manually following the methodology of the work and needs to be redone for the scenario to be evaluated.

In [7]:
#2025
combinations = {
    'station1': [665599552,  1015630488, 1016560181, 691182701, 691182760, 665756836, 665756852, 691347646,  695561219, 1003228795, 922242,    695207075, 691183031,  431377928, 1016159665, 1005343001, 1008016842, 1016257594],
    'station2': [1006733571, 1003326657, 1016343750, 431377898, 4861469,   6556752,   441068146, 1015804915, 431377910, 691183082,  665643896, 922234,    1016098542, 665596960, 1008779110, 690905874,  690905866,  1016093370],
    'station3': [431377880,  691182728,  1004211527, None,      None,      None,      None,      None,       None,      None,       None,      None,      None,       None,      None,       None,       None,       None      ],
    'station4': [None,       1015098069, None,       None,      None,      None,      None,      None,       None,      None,       None,      None,      None,       None,      None,       None,       None,       None      ]
}
multi_tenant_towers = pd.DataFrame(combinations)

#Runs the grouping function
for index, row in multi_tenant_towers.iterrows():
    stations = [int(value) if pd.notnull(value) else None for value in row.values] #Ignore the 'Nones', they only exist so that the list has the same dimensions.
    stations = [station for station in stations if station is not None] #Remove the 'Nones' before creating the list of grouped stations.
    df_stations_info = group_stations(df_stations_info, *stations)

## **Plots**

The cells below plots the polar graphs of the coverage area of the stations and save it to a .zip file.

In [8]:
def plot(azimuths, half_power_angles, filename, results_path):
    #Configures all bars with size 1, causes the visualization of the ERB sector coverage area to be filled from zero to the edge of the graph.
    height_values = [1] * len(azimuths)

    #Convert angles from degrees to radians to be plotted.
    azimuths_rad = np.deg2rad(azimuths)
    half_power_angles_rad = np.deg2rad(half_power_angles)

    #Establish the polar plot.
    plt.figure()
    ax = plt.subplot(111, polar=True)

    #Plot coverage areas.
    ax.bar(x=azimuths_rad, height=height_values, width=half_power_angles_rad, color='#FFCE00', edgecolor='#D69400', alpha=0.3)

    #Configure axis limits.
    angles = [0, np.pi/2, np.pi, 3*np.pi/2]
    for angle in angles:
        ax.plot([0, angle], [0, 1], color='black')
        plt.gcf().set_facecolor("none")

    #Plot the red azimuth lines.
    for azimuth_rad in azimuths_rad:
        ax.plot([0, azimuth_rad], [0, 1], color='red', linewidth='2')
        plt.gcf().set_facecolor("none")

    #Adds 'azimuths_rad' and 'angles'.
    angles.extend(azimuths_rad)
    #Replaces '360' values with '0' in the azimuth list so there is no overlap in the image.
    for i in range(len(azimuths)):
        if azimuths[i] == 360:
            azimuths[i] = 0
    #Create azimuth labels.
    labels = [0, 90, 180, 270]
    labels.extend(azimuths)

    #Plot settings.
    ax.set_ylim(0, 1)
    ax.set_xticks(angles)
    ax.set_xticklabels(labels, fontsize=20, weight='bold')
    for tick in ax.xaxis.get_major_ticks():
        tick.set_pad(15)
    ax.set_yticklabels([])
    ax.set_theta_direction(-1)
    ax.set_theta_zero_location('N')
    ax.grid(False)
    ax.spines['polar'].set_linewidth(2)

    os.makedirs(results_path, exist_ok=True)
    complete_path = os.path.join(results_path, filename)
    
    #Save the figure locally.
    plt.savefig(complete_path, format='png', dpi=600, transparent=True)
    plt.close()

In [9]:
#List to store generated file names.
filenames = []

#Generate and save graphs.
for station, azimuths, half_power_angles in df_stations_info:
    filename = f'{station}.png' #File name.
    plot(azimuths, half_power_angles, filename, results_path)
    filenames.append(filename)

#Create and save the .zip file.
zip_filename = f"azimuth_analysis_{year}.zip"
zip_filepath = os.path.join(results_path, zip_filename)

with zipfile.ZipFile(zip_filepath, "w") as zipf:
    for filepath in filenames:
        zipf.write(results_path +'/' + filepath, os.path.basename(filepath))